In [ ]:
from IPython.display import clear_output, display

import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('max_colwidth', 400)
import numpy as np

from selenium import webdriver

import re
import time
import mouse
import os
from os.path import exists
from win32api import GetSystemMetrics

# Импорт функций из модуля
from ya_maps_parsing import driver_screen_calibration,\
                            mouse_placer_decorator,\
                            decrease_win_scale,\
                            gather_all_snippets,\
                            zoom_value_calibration,\
                            parse_data_within_rect

# Корректировку параметров, если надо, делаем в ya_maps_parsing
from ya_maps_parsing import WIDTH,\
                            HEIGHT,\
                            DRIVER_WIDTH,\
                            DRIVER_HEIGHT,\
                            WIN_POSITION_X,\
                            WIN_POSITION_Y,\
                            WIN_WIDTH,\
                            WIN_HEIGHT

## Запуск драйвера

In [ ]:
# Настраиваем опции драйвера
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")

# Подключаем драйвер
driver = webdriver.Chrome(options=options)
driver.set_window_size(WIN_WIDTH, 
                       WIN_HEIGHT)

# Переходим на сайт настраиваем окно
url = 'https://yandex.ru/maps/'
driver.set_window_position(WIN_POSITION_X, 
                           WIN_POSITION_Y, 
                           windowHandle='current')

# Переходим на страницу яндекс карт
driver.get(url)

# Уменьшим масштаб окна
mouse_placer_decorator(WIDTH)(decrease_win_scale)(iterations=5)

# driver.execute_script("document.body.style.zoom='50%'")

## Калибровка и парсинг

In [ ]:
# Калибровка для определения оптимального значения зума
lon_lat = [37.6, 55.75]      # Примерные координаты Мск
init_zoom = 12
brand_name = 'Магнит'

zoom_value_calibration(driver=driver,
                       lon_lat=lon_lat,
                       init_zoom=init_zoom,
                       brand_name=brand_name)

In [ ]:
# Парсим информацию о точках с яндекс карт
brand_name = 'Магнит'
topleft_lon_lat = [37.6, 55.75]
bottomright_lon_lat = [39, 55]
delta_coef = 0.8
zoom = 12
file_name = 'sample_data.csv'

parse_data_within_rect(driver=driver,
                       topleft_lon_lat=topleft_lon_lat,
                       bottomright_lon_lat=bottomright_lon_lat,
                       delta_coef=delta_coef,
                       zoom=zoom,
                       brand_name=brand_name,
                       file_name=file_name)

## Проверка парсинга с текущего положения карты

In [ ]:
# Сбор данных с текущего положения карты
test_brand_df = gather_all_snippets(driver=driver)
test_brand_df

## Прочее

In [ ]:
# Очищаем и вбиваем значения (сеть + адрес + координаты)
# Индекс в ДФ
df_index = 0

# Текст запроса
text_to_send = 'Магнит ' + ' '.join(geo_df.iloc[df_index].tolist())
print(text_to_send)

# Ищем конкретную точку
# search_query(text_to_send)


In [ ]:
# Ищем сеть
search_query(driver=driver, text='Магнит')
time.sleep(2)

# Собираем данныaе с текущего положения карты
magnit_msk_df = gather_all_snippets(driver=driver)

In [ ]:
res = pd.DataFrame(columns=magnit_msk_df.columns)

In [ ]:
b = res.assign(res_cnt=1)\
        .merge(magnit_msk_df.assign(cnt=1), 
               on=['adress', 'lat', 'lon'], 
               how='right')
print(len(b[b['res_cnt'].isna()].drop_duplicates()))
res = pd.concat([res, magnit_msk_df])
cnt = res.copy()
cnt['cnt'] = 1
a = cnt.groupby(by=['adress', 'lat', 'lon'], as_index=False).count()
print(len(a))

a.hist('cnt')

In [ ]:
current_url = driver.current_url
new_value = 8
filter_text = 'z={}'
get_function = get_zoom_value

replace_filter_value(url=current_url,
                     new_value=new_value,
                     filter_text=filter_text,
                     get_function=get_function)

In [ ]:
current_url = driver.current_url
new_value = [22, 11]
filter_text = 'll={}%2C{}'
get_function = get_coordinates

replace_filter_value(url=current_url,
                     new_value=new_value,
                     filter_text=filter_text,
                     get_function=get_function)

In [ ]:
# Переход на нужную координату

lon_lat = [39, 55]
zoom = 12

# Определяем URL для стартовой точки
start_point_url = move_to_url(driver=driver, lon_lat=lon_lat)
# Переходим на стартовую точку
driver.get(start_point_url)